In [60]:
# load the demographic data
import pandas as pd
url = "https://data.cityofnewyork.us/resource/vmmu-wj3w.csv?$limit=1000000"
df = pd.read_csv(url)


In [61]:
# add district and boro info
boros = {"K":"Brooklyn", "X":"Bronx", "M": "Manhattan", "Q": "Queens", "R": "Staten Island"}

def district(dbn): return int(dbn[:2])
def boro(dbn): return boros[dbn[2]]

df["district"] = df["dbn"].apply(district)
df["boro"] = df["dbn"].apply(boro)

# just the most recent year
df = df[ df["year"] == "2020-21" ]



In [62]:
def str_pct(row):
    pct = row["poverty_1"][:-1]
    try:
        pct = float(pct) / 100
    except:
        if "Above" in pct:
            pct = row["total_enrollment"] * .96
        elif "Below" in pct:
            pct = row["total_enrollment"] * .04
    return float(pct)

df["poverty_1"] = df.apply(str_pct, axis = 1)

df["poverty_1"]

4         0.819
9         0.712
14        0.709
19      262.080
24        0.769
         ...   
9148    399.360
9153      0.832
9158      0.878
9163      0.819
9168      0.864
Name: poverty_1, Length: 1878, dtype: float64

In [34]:
# the columns we want to look at
cols = ['dbn', 
        'district',
        'boro',
        'school_name', 
        'total_enrollment',
        'female_1',
        'male_1',
        'asian_1', 
        'black_1', 
        'hispanic_1', 
        'multi_racial_1', 
        'native_american_1', 
        'white_1', 
        'students_with_disabilities_1', 
        'english_language_learners_1',  
        'poverty_1']

df[cols]
# df["boro"]

,dbn,district,boro,school_name,total_enrollment,asian_1,black_1,hispanic_1,multi_racial_1,native_american_1,white_1,students_with_disabilities_1,english_language_learners_1,poverty_1
4,01M015,1,Manhattan,P.S. 015 Roberto Clemente,193,0.135000,0.275000,0.528000,0.005000,0.000000,0.057000,0.223000,0.109,81.9%
9,01M019,1,Manhattan,P.S. 019 Asher Levy,212,0.061000,0.193000,0.613000,0.033000,0.005000,0.080000,0.392000,0.042,71.2%
14,01M020,1,Manhattan,P.S. 020 Anna Silver,412,0.248000,0.133000,0.522000,0.015000,0.007000,0.073000,0.218000,0.119,70.9%
19,01M034,1,Manhattan,P.S. 034 Franklin D. Roosevelt,273,0.026000,0.381000,0.557000,0.004000,0.004000,0.029000,0.392000,0.062,Above 95%
24,01M063,1,Manhattan,The STAR Academy - P.S.63,208,0.029000,0.192000,0.635000,0.034000,0.005000,0.091000,0.279000,0.014,76.9%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9148,84X705,84,Bronx,Family Life Academy Charter School,416,0.007212,0.211538,0.776442,0.000000,0.002404,0.000000,0.100962,0.197,Above 95%
9153,84X706,84,Bronx,Harriet Tubman Charter School,647,0.000000,0.616692,0.358578,0.006182,0.012365,0.006182,0.097372,0.110,83.2%
9158,84X717,84,Bronx,Icahn Charter School,328,0.012195,0.500000,0.478659,0.000000,0.009146,0.000000,0.070122,0.076,87.8%
9163,84X718,84,Bronx,Bronx Charter School for Better Learning,570,0.008772,0.845614,0.100000,0.015789,0.014035,0.014035,0.108772,0.012,81.9%


In [63]:
public = df[df["district"] < 75]
charter = df[df["district"] == 84]


In [65]:
p_schools = df[ df["poverty_1"] > .2] 

In [67]:
print("charter", charter["total_enrollment"].mean())
print(public["total_enrollment"].mean())

23.982955056179776


34.99776718547341